In [1]:
import json
import pandas as pd
import numpy as np
import os

os.environ['HF_HOME'] = '/net/tscratch/people/plgkonkie311/cache/'

In [2]:
cd spatial

/net/tscratch/people/plgkonkie311/spatial


In [3]:
path = 'booksy_wroclaw.json'

with open(path, 'r') as file:
    data = json.load(file)


In [4]:
df_booksy = pd.DataFrame(data)

In [ ]:
df_booksy.head()

,id,name,address,rating,services
0,0,Barbershop Crazy Razor,"bulwar Ikara 23a/1, 54-130, Wrocław, Fabryczna","4,9","[{'price': '20,00 zł', 'name': 'wosk'}, {'pric..."
1,1,Barbershop Crazy Razor,"Legnicka 59/U14, 54-203, Wrocław, Fabryczna","5,0","[{'price': '499,00 zł', 'name': 'terapia gen f..."
2,2,Klinika Diagnostyki Skóry by Bellissimki,"Legnicka 72, 54-206, Wrocław, Fabryczna","5,0","[{'price': '20,00 zł', 'name': 'konsultacja ko..."
3,3,Instytut Urody Fantastic Body,"UL.LEGNICKA 58, 54-204, Wrocław, Fabryczna","4,9","[{'price': '79,00 zł', 'name': 'strzyżenie chł..."
4,4,Fryzjerek Magnolia,"Wagonowa 2d, 53-609, Wrocław, Fabryczna","5,0","[{'price': '100,00 zł', 'name': 'strzyżenie kl..."


In [6]:
# import json
# from opencage.geocoder import OpenCageGeocode
# from tqdm import tqdm

# # Replace with your OpenCage API key
# API_KEY = "8b20f361ab754e0ca2e82cc12c69aed8"
# geocoder = OpenCageGeocode(API_KEY)

# def geocode_address(row):
#     """Geocodes a single address from a DataFrame row."""
#     address = row['address']
#     result = geocoder.geocode(address)
#     if result:
#         return {
#             'id': row['id'],
#             'address': address,
#             'lat': result[0]['geometry']['lat'],
#             'lng': result[0]['geometry']['lng'],
#             'rating': row['rating'],
#             'services': row['services'],
#         }
#     else:
#         return {
#             'id': row['id'],
#             'address': address,
#             'lat': None,
#             'lng': None,
#             'rating': row['rating'],
#             'services': row['services'],
#         }

# def save_results_to_file(results, filename='geocoding_results.json'):
#     """Saves the geocoding results to a JSON file."""
#     with open(filename, 'w') as file:
#         json.dump(results, file, indent=4)

# def process_geocoding(df, output_file='geocoding_results.json'):
#     """Processes geocoding for all rows in the DataFrame and saves results to a file."""
#     results = []
#     for _, row in tqdm(df.iterrows(), total=len(df)):
#         result = geocode_address(row)
#         results.append(result)
#         save_results_to_file(results, output_file)  # Save results after each row for data integrity
#     return results



In [7]:
# results = process_geocoding(df_booksy)

In [8]:
import json

# Wczytaj GeoJSON
with open("export.geojson", "r", encoding="utf-8") as file:
    geojson_data = json.load(file)

In [9]:

# Funkcja przetwarzająca dane
def preprocess_geojson(geojson):
    objects = []
    for feature in geojson.get("features", []):
        properties = feature.get("properties", {})
        coordinates = feature.get("geometry", {}).get("coordinates", [])

        cleaned_properties = properties.copy()

        del cleaned_properties["@id"]
        rest_data_str = json.dumps(cleaned_properties)

        # Pobierz kluczowe pola
        obj = {
            "name": properties.get("name", "Unknown"),
            "amenity": properties.get("amenity", "Unknown"),
            "address": properties.get("addr:street", "Unknown") + " " + properties.get("addr:housenumber", "") + ", " + properties.get("addr:city", "Unknown"),
            "lattitude": coordinates[1],
            "longitude": coordinates[0],
            "rest_data": rest_data_str
        }
        objects.append(obj)
    return objects

# Przetwórz dane
processed_data = preprocess_geojson(geojson_data)


In [10]:
# to dataframe

df_geojson = pd.DataFrame(processed_data)

In [11]:
df_geojson = df_geojson[df_geojson['amenity'] != 'bench']
df_geojson = df_geojson[df_geojson['amenity'] != 'Unknown']
df_geojson = df_geojson[df_geojson['amenity'] != 'waste_basket']
df_geojson = df_geojson[df_geojson['name'] != 'Unknown']

df_geojson['amenity'].value_counts()


amenity
fast_food         718
restaurant        470
cafe              261
pharmacy          236
bicycle_rental    219
                 ... 
townhall            1
bus_station         1
baby_hatch          1
hospital            1
nursing_home        1
Name: count, Length: 83, dtype: int64

In [12]:
df_cafe = df_geojson[df_geojson['amenity'] == 'cafe']
df_cafe

,name,amenity,address,lattitude,longitude,rest_data
28,Larks,cafe,"Kotlarska 32A, Wrocław",51.111868,17.032191,"{""addr:city"": ""Wroc\u0142aw"", ""addr:housenumbe..."
30,Czekoladziarnia,cafe,"Więzienna 31a, Wrocław",51.111592,17.032107,"{""addr:city"": ""Wroc\u0142aw"", ""addr:housenumbe..."
39,Kalambur,cafe,"Kuźnicza 29a, Wrocław",51.112828,17.034624,"{""addr:city"": ""Wroc\u0142aw"", ""addr:housenumbe..."
46,Sawara,cafe,"Unknown , Unknown",51.083968,17.037038,"{""amenity"": ""cafe"", ""name"": ""Sawara"", ""wheelch..."
290,Mleczarnia,cafe,"Pawła Włodkowica 5, Wrocław",51.108151,17.025399,"{""addr:city"": ""Wroc\u0142aw"", ""addr:housenumbe..."
...,...,...,...,...,...,...
18424,SweetLand Cafe,cafe,"Unknown , Unknown",51.112292,17.032542,"{""amenity"": ""cafe"", ""name"": ""SweetLand Cafe"", ..."
18426,Pi Kawka,cafe,"Unknown , Unknown",51.111640,17.055902,"{""amenity"": ""cafe"", ""cuisine"": ""breakfast"", ""n..."
18427,ArtCaf,cafe,"Unknown , Unknown",51.109999,17.013002,"{""amenity"": ""cafe"", ""name"": ""ArtCaf"", ""start_d..."
18428,Coccoli,cafe,"Unknown , Unknown",51.149092,16.867147,"{""amenity"": ""cafe"", ""cuisine"": ""ice_cream"", ""n..."


In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

/net/tscratch/people/plgkonkie311/miniconda3/envs/hallu/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-01-27 14:22:07.343644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737984127.495487 3443016 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737984127.554972 3443016 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 14:22:07.843849: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary i

model.safetensors:  83%|########3 | 933M/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [14]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [15]:
from uuid import uuid4

from langchain_core.documents import Document

documents = []
uuids = []

for _, row in df_geojson.iterrows():
    doc = Document(
        page_content=row['rest_data'],
        metadata={
            'amenity': row['amenity'],
            "name": row['name'],
            "address": row['address'],
            "lattitude": row['lattitude'],
            "longitude": row['longitude'],
        }
    )
    documents.append(doc)
    uuids.append(str(uuid4()))

vector_store.add_documents(documents=documents, ids=uuids)

['ecda8c3c-c7fc-4f71-8a2a-8a21636debd9',
 '306bc7ba-d9b7-463a-ba68-1328260b6aa9',
 '0ff39950-9f13-48a0-8808-f2d3fe616f86',
 'dbfd66e5-0f32-46c7-bd8b-a7154594b582',
 'b0cf67bd-6c8a-4e7d-9350-d68295e7f166',
 '916922d8-f267-4a6d-9697-2fd81fb9d583',
 '5e0db92a-4d07-4c35-aab6-c29c4485ce0c',
 'bf0b2b27-4f40-43df-8f4b-4f7badaf34fa',
 '7579fcac-de5c-4f6d-978e-1d627165ae89',
 '26b6924a-de17-4e2e-9377-aae860e7f171',
 '82af8257-01dc-4b8d-acc7-0b152d1cc103',
 'efb0f988-5fca-430e-9401-a1a053e21749',
 '81862c87-b54e-42c3-9bfd-6862dd589bcc',
 'f7c14653-684e-4efd-b91a-1d41f52b283e',
 '0a0ff160-6fe8-42ba-8839-559d78616f7e',
 '35a56708-0cbd-487a-ace6-e59f7617cf42',
 '38fdec76-e4e5-4e81-b6fb-5349abd667db',
 'ed2feec4-c56a-4481-bb9c-687c7e904cd6',
 '9b3a8d4e-63cf-499a-964d-1d60202b469a',
 'fcb52e97-689f-40bb-89d4-9d0b297a7f37',
 '569f44b6-bedd-407c-86b1-6b0780680a16',
 '7c9d1504-c324-4b8e-8ba2-bd68e88fbeab',
 'b2afa5f7-cf0d-431e-be8a-776da0b85198',
 '6d18569c-6e3b-4d0d-a64b-d59ebc77ef0e',
 '2c2be589-dba9-

In [16]:
vector_store.save_local('all_wroclaw_new')

In [17]:
results = vector_store.similarity_search(
    "'name': 'Dworzec Główny', 'description': 'Wrocław Główny, Dworzec kolejowy'",
    k=5,
    # filter={"amenity": "cafe"}
)
for res in results:
    print(f"* [{res.metadata}] {res.page_content}")

* [{'amenity': 'bus_station', 'name': 'Dworzec Autobusowy Wrocław', 'address': 'Unknown , Unknown', 'lattitude': 51.0966307, 'longitude': 17.0362768}] {"amenity": "bus_station", "image": "https://fotopolska.eu/1142/1142149/686/Wroclaw_Galeria_handlowa_Wroclavia.jpg", "image:2": "https://polska-org.pl/8187778,foto.html", "level": "-2", "name": "Dworzec Autobusowy Wroc\u0142aw", "name:en": "Bus Station Wroclaw", "name:pl": "Dworzec Autobusowy Wroc\u0142aw", "name:ru": "\u0410\u0432\u0442\u043e\u0432\u043e\u043a\u0437\u0430\u043b \u0412\u0440\u043e\u0446\u043b\u0430\u0432", "name:uk": "\u0410\u0432\u0442\u043e\u0432\u043e\u043a\u0437\u0430\u043b \u0412\u0440\u043e\u0446\u043b\u0430\u0432", "opening_hours": "24/7", "operator": "Polbus PKS", "start_date": "2017-11-07", "url": "https://polska-org.pl/6933414,Wroclaw,Dworzec_autobusowy_Wroclaw_Polbus_PKS.html", "website": "https://dworzecwroclaw.pl/", "wheelchair": "yes", "wikidata": "Q42799456", "wikipedia": "pl:Dworzec Wroc\u0142aw"}
* [{'am

In [18]:
center = results[1].metadata['lattitude'], results[1].metadata['longitude']

def generate_square_filter(center, radius_km=5, amenity=None):
    lat, lon = center
    
    # Approximate degrees per kilometer for latitude and longitude
    lat_delta = radius_km / 110.574  # 1 degree latitude = 110.574 km
    lon_delta = radius_km / (111.320 * np.cos(np.radians(lat)))  # 1 degree longitude = 111.320 km * cos(latitude)

    # Create the bounding box as a filter
    filter_query = {
        "$and": [
            {"lattitude": {"$gte": lat - lat_delta}},  # min latitude
            {"lattitude": {"$lte": lat + lat_delta}},  # max latitude
            {"longitude": {"$gte": lon - lon_delta}},  # min longitude
            {"longitude": {"$lte": lon + lon_delta}},  # max longitude
        ]
    }
    if amenity:
        filter_query["$and"].append({"amenity": amenity})
    return filter_query

generate_square_filter(center, radius_km=5)



{'$and': [{'lattitude': {'$gte': 51.05612671335214}},
  {'lattitude': {'$lte': 51.146563886647854}},
  {'longitude': {'$gte': 16.951574620693037}},
  {'longitude': {'$lte': 17.094630379306963}}]}

In [19]:
results = vector_store.similarity_search(
    "'description': 'Polska kuchnia', 'cuisine': 'polish'",
    k=5,
    filter=generate_square_filter(center, radius_km=3, amenity='restaurant')
)
for res in results:
    print(f"* [{res.metadata}] {res.page_content}")

* [{'amenity': 'restaurant', 'name': 'Pałacyk', 'address': 'Unknown , Unknown', 'lattitude': 51.1026472, 'longitude': 17.0339506}] {"amenity": "restaurant", "cuisine": "polish", "name": "Pa\u0142acyk"}
* [{'amenity': 'restaurant', 'name': 'Prosto z Patelni', 'address': 'Unknown , Unknown', 'lattitude': 51.121986, 'longitude': 16.9892579}] {"amenity": "restaurant", "cuisine": "polish", "name": "Prosto z Patelni"}
* [{'amenity': 'restaurant', 'name': 'Żak', 'address': 'Unknown , Unknown', 'lattitude': 51.1106027, 'longitude': 17.0315414}] {"amenity": "restaurant", "cuisine": "polish", "name": "\u017bak", "outdoor_seating": "yes"}


In [20]:
# visualisation of results on map

import folium

m = folium.Map(location=center, zoom_start=15)

for res in results:
    folium.Marker(
        location=[res.metadata["lattitude"], res.metadata["longitude"]],
        popup=res.metadata["name"],
    ).add_to(m)

folium.Marker(
    location=center,
    popup="Wrocław Główny",
    icon=folium.Icon(color="red"),
).add_to(m)


display(m)


In [21]:
with open('geocoding_results.json', 'r') as file:
    data = json.load(file)

with open('booksy_wroclaw.json', 'r') as file:
    data_w_name = json.load(file)


booksy_df = pd.DataFrame(data)
booksy_df.head()

booksy_df_2 = pd.DataFrame(data_w_name)
booksy_df_2 = booksy_df_2[['id', 'name']]
booksy_df_2.head()


booksy_df = pd.merge(booksy_df, booksy_df_2, on='id', how='left')
booksy_df

,id,address,lat,lng,rating,services,name
0,0,"bulwar Ikara 23a/1, 54-130, Wrocław, Fabryczna",51.100000,17.033300,"4,9","[{'price': '20,00 zł', 'name': 'wosk'}, {'pric...",Barbershop Crazy Razor
1,1,"Legnicka 59/U14, 54-203, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '499,00 zł', 'name': 'terapia gen f...",Barbershop Crazy Razor
2,2,"Legnicka 72, 54-206, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '20,00 zł', 'name': 'konsultacja ko...",Klinika Diagnostyki Skóry by Bellissimki
3,3,"UL.LEGNICKA 58, 54-204, Wrocław, Fabryczna",51.100000,17.033300,"4,9","[{'price': '79,00 zł', 'name': 'strzyżenie chł...",Instytut Urody Fantastic Body
4,4,"Wagonowa 2d, 53-609, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '100,00 zł', 'name': 'strzyżenie kl...",Fryzjerek Magnolia
...,...,...,...,...,...,...,...
2286,2286,"Długa 73, 49-306, Brzeg",50.861981,17.466747,"5,0","[{'price': '150,00 zł', 'name': 'manicure żelo...",Aleksandra Rabiej Beauty Artist
2287,2287,"Sucha 1, 50-086, Wrocław, Krzyki",51.100000,17.033300,"4,9","[{'price': '229,00 zł 206,10 zł', 'name': 'mak...",Imagine Makeup Małgorzata Kowalkowska
2288,2288,"plac Nowy Targ 28, 50-141, Wrocław",51.110081,17.038434,"4,5","[{'price': '299,00 zł', 'name': 'serwis bateri...",INGLOT Wroclavia Wrocław
2289,2289,"Świętokrzyska 38/1B, 50-327, Wrocław, Śródmieście",51.100000,17.033300,"5,0",[],Malowana Lala Salon Urody


In [22]:
# drop nan values
booksy_df = booksy_df.dropna(subset=['lat', 'lng'])
booksy_df

,id,address,lat,lng,rating,services,name
0,0,"bulwar Ikara 23a/1, 54-130, Wrocław, Fabryczna",51.100000,17.033300,"4,9","[{'price': '20,00 zł', 'name': 'wosk'}, {'pric...",Barbershop Crazy Razor
1,1,"Legnicka 59/U14, 54-203, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '499,00 zł', 'name': 'terapia gen f...",Barbershop Crazy Razor
2,2,"Legnicka 72, 54-206, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '20,00 zł', 'name': 'konsultacja ko...",Klinika Diagnostyki Skóry by Bellissimki
3,3,"UL.LEGNICKA 58, 54-204, Wrocław, Fabryczna",51.100000,17.033300,"4,9","[{'price': '79,00 zł', 'name': 'strzyżenie chł...",Instytut Urody Fantastic Body
4,4,"Wagonowa 2d, 53-609, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '100,00 zł', 'name': 'strzyżenie kl...",Fryzjerek Magnolia
...,...,...,...,...,...,...,...
2286,2286,"Długa 73, 49-306, Brzeg",50.861981,17.466747,"5,0","[{'price': '150,00 zł', 'name': 'manicure żelo...",Aleksandra Rabiej Beauty Artist
2287,2287,"Sucha 1, 50-086, Wrocław, Krzyki",51.100000,17.033300,"4,9","[{'price': '229,00 zł 206,10 zł', 'name': 'mak...",Imagine Makeup Małgorzata Kowalkowska
2288,2288,"plac Nowy Targ 28, 50-141, Wrocław",51.110081,17.038434,"4,5","[{'price': '299,00 zł', 'name': 'serwis bateri...",INGLOT Wroclavia Wrocław
2289,2289,"Świętokrzyska 38/1B, 50-327, Wrocław, Śródmieście",51.100000,17.033300,"5,0",[],Malowana Lala Salon Urody


In [23]:
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

booksy_vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [24]:
from uuid import uuid4

from langchain_core.documents import Document

documents = []
uuids = []

for _, row in booksy_df.iterrows():
    page_content = 'Name: ' + row['name'] + ' Rating: ' + str(row['rating']) + ' Address: ' + row['address']
    page_content = f'{page_content}  Services: {row["services"]}'

    doc = Document(
        page_content=page_content,
        metadata={
            'name': row['name'],
            'rating': row['rating'],
            'lattitude': float(row['lat']),
            'longitude': float(row['lng']),
            'address': row['address'],
        }
    )

    documents.append(doc)
    uuids.append(str(uuid4()))

booksy_vector_store.add_documents(documents=documents, ids=uuids)

['fcc62377-40f1-4206-b871-5d20d86c2f4f',
 'f63f8669-a6cf-43d9-8229-e3fa7d2a382e',
 '67b4159f-d15c-4c26-aa65-9b0809ee5334',
 '1aec1f97-ac00-48ef-9047-abb6ce891c43',
 'debd5837-eb83-4246-a392-791ae539ca95',
 '711df407-f9c4-48e2-aeaa-378daf55cfc0',
 'd459d4e7-baa2-4894-aecf-ba900ef6fac5',
 '4fa78575-2f8a-4d36-b545-7e421cc873ae',
 'e3bcd46e-2d40-4ba5-b1d6-b8c06b20e739',
 '90700be4-17ce-40b0-92ec-8097dd56b51b',
 '50aeb52e-e6a6-4071-b95d-021c5433a095',
 'fa200f67-6d3c-4e04-a81c-b886a5712d20',
 '667502f5-e5dc-4752-b0da-49a4352f8c10',
 '8427aea7-e9b1-4bc0-91b1-c410c53cc96c',
 '9d4704be-3ee6-4f39-a703-93e752b44da2',
 '06101db2-ee81-47bf-8514-7470f6ec1ec3',
 'dd4c00c0-bf82-48b2-b662-4e5ad681f887',
 '6cb06e7c-a13e-4e45-862f-ee6c2295df43',
 '5850f9a8-09e3-43f0-b814-708d9d434dda',
 '54d2e6ee-fe61-4167-bd0d-41c7536fb441',
 'bc2d314a-3a81-410b-9785-ee7b839f3664',
 '24e3add5-b099-4bb7-aff7-072633abdb05',
 'd37c243f-9960-48cb-9b44-8d669d709abf',
 'ec34d151-7d83-45ee-aae9-46b47f83de59',
 'ebc46ff1-8a20-

In [25]:
booksy_vector_store.save_local('booksy_wroclaw_new')

In [26]:
results = booksy_vector_store.similarity_search(
    "Fryzjer oferujący strzyżenie męskie do 70 zł",
    k=5,
    filter=generate_square_filter(center, radius_km=1)
)

for res in results:
    print(f"* [{res.metadata}] {res.page_content}")

* [{'name': 'Studio71 MB Fryzjerskie', 'rating': '5,0', 'lattitude': 51.1, 'longitude': 17.03333, 'address': 'ulica Pomorska 55, 34u, Wrocław'}] Name: Studio71 MB Fryzjerskie Rating: 5,0 Address: ulica Pomorska 55, 34u, Wrocław  Services: [{'price': '70,00 zł', 'name': 'strzyżenie klasyczne'}, {'price': '100,00 zł', 'name': 'combo strzyżenie klasyczne trymowanie brody'}, {'price': '50,00 zł', 'name': 'strzyżenie maszynką'}, {'price': '70,00 zł', 'name': 'strzyżenie klasyczne'}, {'price': '50,00 zł', 'name': 'strzyżenie maszynką'}, {'price': '80,00 zł', 'name': 'strzyżenie nożyczkami'}, {'price': '80,00 zł', 'name': 'strzyżenie skin fade'}, {'price': '100,00 zł', 'name': 'combo strzyżenie klasyczne trymowanie brody'}, {'price': '110,00 zł', 'name': 'combo strzyżenie klasyczne broda spa'}, {'price': '120,00 zł', 'name': 'strzyżenie skin fade broda spa'}, {'price': '60,00 zł', 'name': 'trymowanie brody'}, {'price': '70,00 zł', 'name': 'broda spa'}, {'price': '50,00 zł', 'name': 'strzyżeni

In [27]:
import random

m = folium.Map(location=center, zoom_start=15)

for res in results:
    lat_offset = random.uniform(-0.0005, 0.0005)
    lng_offset = random.uniform(-0.0005, 0.0005)

    lat = res.metadata["lattitude"] + lat_offset
    lng = res.metadata["longitude"] + lng_offset
    
    folium.Marker(
        location=[lat, lng],
        popup=res.metadata["name"],
    ).add_to(m)

folium.Marker(
    location=center,
    popup="Wrocław Główny",
    icon=folium.Icon(color="red"),
).add_to(m)

display(m)